In [1290]:
'''
Created on 2016. 6. 12

@author: dato
@desc: cnlp hw11 (N-Gram Models)
'''
#-*- coding: utf-8 -*-

import io
import numpy as np

In [1291]:
# data_file = './ALICES_ADVENTURES_ IN_WONDERLAND.txt'
data_file = './Pride_and_Prejudice.txt'

In [1292]:
f = io.open(data_file, 'r', encoding='utf-8')
data = f.read()

# pre-processing

In [1293]:
# strip, remove newline, remove double spacing
data = data.strip()
data = data.replace("\n", " ")
data = data.replace("  ", " ")
data = data.replace("  ", " ")
data = data.replace("  ", " ")
data = data.replace("  ", " ")
data = data.replace("  ", " ")
data = data.lower()
len(data)

700801

In [1294]:
def cal_doc_dic(data, n):

    doc_dic = {}
    buffer = []
    
    for i in range(n):
        buffer.append('a')   #dummy data
    
    for char in data:
        
        str_char = str(char)

        buffer[:] = buffer[1:]   #shift-n
        buffer.append(str_char)
        key = "".join(buffer)
        
#         print buffer
        
        if doc_dic.has_key(key):
            doc_dic[key] += 1
#             break
        else:
            doc_dic[key] = 1            

    print "# of vocabulary in n-gram : %d" % len(doc_dic)
    
    return doc_dic

In [1305]:
def norm_doc_dic(dic_n, dic_n_1, n):

    new_dic = {}
    
    if n is 1:
        for key in dic_n.keys():
            new_dic[key] =  dic_n[key] / float(sum(dic_n.values()))
        
    else:
        for key in dic_n.keys():
            if dic_n_1.has_key(key[:-1]) :
                new_dic[key] =  dic_n[key] / float(dic_n_1[key[:-1]])
    
    return new_dic

In [1306]:
def log_doc_dic(dic):
    
    new_dic = {}
        
    for key in dic.keys():
        new_dic[key] =  np.log(dic[key])
        
    return new_dic

In [1307]:
def check_prob(ngram_nom, char_list, data):
    prob = 0
    
    list_char = list(char_list)
    
    for char in list_char:
        
        comp_char = data+char
#         print comp_char
        
        if ngram_nom.has_key(comp_char):
            prob += ngram_nom[comp_char]
#             print ngram_nom[comp_char]
    
    return prob

In [1308]:
doc_dic = cal_doc_dic2(data, 1)

# of dictionary in data : 27


In [1309]:
# remove unwanted char from data

char_list = {'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' '}
target_dic = dict.fromkeys(char_list, 1)
len(target_dic)

for char in doc_dic.keys():
    if target_dic.has_key(char):
        continue
    else:        
        data = data.replace(char, "") 

# calcuate n-gram dictionary

In [1310]:
unigram_cnt = cal_doc_dic(data, 1)
bigram_cnt = cal_doc_dic(data, 2)
trigram_cnt = cal_doc_dic(data, 3)
quadgram_cnt = cal_doc_dic(data, 4)
pentagram_cnt = cal_doc_dic(data, 5)

# of vocabulary in n-gram : 27
# of vocabulary in n-gram : 519
# of vocabulary in n-gram : 4197
# of vocabulary in n-gram : 16877
# of vocabulary in n-gram : 47406


In [1311]:
unigarm_nom = norm_doc_dic(unigram_cnt, unigram_cnt, 1)
bigram_nom = norm_doc_dic(bigram_cnt, unigram_cnt, 2)
trigram_nom = norm_doc_dic(trigram_cnt, bigram_cnt, 3)
quadgram_nom = norm_doc_dic(quadgram_cnt, trigram_cnt, 4)
pentagram_nom = norm_doc_dic(pentagram_cnt, quadgram_cnt, 5)


ngram_nom = []
ngram_nom.append(unigram_nom)
ngram_nom.append(bigram_nom)
ngram_nom.append(trigram_nom)
ngram_nom.append(quadgram_nom)
ngram_nom.append(pentagram_nom)

In [1312]:
#  check the prob table
print check_prob(bigram_nom, char_list, 'a')
print check_prob(trigram_nom, char_list, 'ab')
print check_prob(quadgram_nom, char_list, 'stu')
print check_prob(pentagram_nom, char_list, 'help')

1.00002343896
1.0
1.0
1.0


In [1313]:
bigram_log = log_doc_dic(bigram_nom)
trigram_log = log_doc_dic(trigram_nom)
quadgram_log = log_doc_dic(quadgram_nom)
pentagram_log = log_doc_dic(pentagram_nom)

# 1. Generate Random Sequence & log prob

In [1314]:
def predic_next_char(ngram_nom, char_list, order, given_char):
    sample = np.random.uniform(0, 1)
    prob = 0
    # print sample
    
    last_char = ""
    
    if order == 1:
        given_char = ""
        
    for char in char_list:
        target = given_char + char
        
        if ngram_nom[order-1].has_key(target):
            prob += ngram_nom[order-1][target]
            last_char = char
            # print char + " " + str(ngram_nom[order-1][target]) + "₩t " + str(prob)
            
        if prob > sample:
            return char, np.log(ngram_nom[order-1][target])
        
    return last_char, np.log(ngram_nom[order-1][target])

In [1315]:
print predic_next_char(ngram_nom, char_list, 1, '')
print predic_next_char(ngram_nom, char_list, 2, 'a')
print predic_next_char(ngram_nom, char_list, 3, 'al')
print predic_next_char(ngram_nom, char_list, 4, 'ali')
print predic_next_char(ngram_nom, char_list, 5, 'alic')

('t', -2.525113374071708)
('d', -2.8584926888377664)
('l', -0.71563583033292399)
('t', -0.53492317534505118)
('i', -0.916290731874155)


In [1316]:
def gen_random(ngram_nom, char_list, order, length):

    sum_ll = 0
    ll = 0

    start_char = ""
    tmp_char = ""
    
    for i in range(1, order):
        tmp_char, ll = predic_next_char(ngram_nom, char_list, i, start_char)
        start_char += tmp_char
        sum_ll += ll
    
    result = ""
    result += start_char
    
    for x in range(length-order+1):
        
        tmp_char, ll = predic_next_char(ngram_nom, char_list, order, start_char)
        start_char = start_char[1:] + tmp_char
        
        result += tmp_char 
        sum_ll += ll
    
    return result, np.exp(sum_ll)

In [1319]:
print gen_random(ngram_nom, char_list, 1, 20)
print gen_random(ngram_nom, char_list, 2, 20)
print gen_random(ngram_nom, char_list, 3, 20)
print gen_random(ngram_nom, char_list, 4, 20)
print gen_random(ngram_nom, char_list, 5, 20)

('pndde b eow  kudot  ', 2.5272825754077405e-25)
('d mero st o t ind ak', 7.6688185952950408e-19)
('h a vok wers thery t', 1.3267908828235493e-16)
('ms thered boutes and', 2.5176159894064899e-17)
('pedit the regain bot', 2.3158665251667929e-15)


In [1320]:
def print_gen(order, length, number):
    print "[order=%d, length=%d]" %(order, length)
    
    for i in range(number):
        sen, ll = gen_random(ngram_nom, char_list, order, length)
        print "[%d-trial] : " %(i+1) + sen + "\t" + str(ll)
    
    print "\n"

In [1326]:
print_gen(1, 20, 5)
print_gen(2, 20, 5)
print_gen(3, 20, 5)
print_gen(4, 20, 5)
print_gen(5, 20, 5)

[order=1, length=20]
[1-trial] : edt icl    hsnaa tl 	1.64166984739e-22
[2-trial] :  letsovnsrwe  tr tin	1.27079359636e-24
[3-trial] :  lloynpawbke trt rtr	3.06431991073e-27
[4-trial] : lirsne u eoocie hptt	7.53597843782e-25
[5-trial] : sm aroi  citenoueart	9.47340352926e-25


[order=2, length=20]
[1-trial] : r his blel lest thar	1.76501575168e-19
[2-trial] :  an tto y wlly id wh	6.50893261067e-20
[3-trial] :  rshat rtowintrsioul	2.05881943886e-22
[4-trial] : shir esarctofr armar	3.65612494329e-23
[5-trial] : gins iclit onanf s b	1.20152342522e-22


[order=3, length=20]
[1-trial] : spink shapprear at w	2.83321864742e-18
[2-trial] : o turhat twom tionev	9.65932771375e-19
[3-trial] : illitat horgivingett	3.37968851833e-19
[4-trial] : cceme dares the stri	2.25494086317e-18
[5-trial] : m hat lialwast hish 	2.25135549865e-17


[order=4, length=20]
[1-trial] : s inford i lastoward	3.61462317953e-15
[2-trial] : and wickham all benn	3.90299802369e-10
[3-trial] : s mr birty to a virt	1.99428397

# complete future character

In [1327]:
def complete_chars(ngram_nom, char_list, order, given_char):

    sum_ll = 0
    ll = 0
    
    start_char = given_char
    tmp_char = ""
    
    match_order = (order-1) - len(given_char)
    
    if match_order > 0:
               
        for i in range(order-match_order, order):
            tmp_char, ll = predic_next_char(ngram_nom, char_list, i, start_char)
            start_char += tmp_char
            sum_ll += ll
            
            if tmp_char is " ":
                return start_char, np.exp(sum_ll)
    

    result = ""
    result += start_char
    
    start_char = start_char[-(order-1):]
    
    for x in range(10000):
        
        tmp_char, ll = predic_next_char(ngram_nom, char_list, order, start_char)
        start_char = start_char[1:] + tmp_char
        
        result += tmp_char 
        sum_ll += ll
        
        if tmp_char is ' ':
            break
    
    return result, np.exp(sum_ll)

In [1329]:
print "unigram"
print complete_chars(ngram_nom, char_list, 1, "tu")
print "bigram"
print complete_chars(ngram_nom, char_list, 2, "tu")
print "trigram"
print complete_chars(ngram_nom, char_list, 3, "tu")
print "quadgram"
print complete_chars(ngram_nom, char_list, 4, "tu")
print "pentagram"
print complete_chars(ngram_nom, char_list, 5, "tu")

unigram
('tua ', 0.01243380312456531)
bigram
('tutrshe ', 1.711925397787971e-06)
trigram
('turpoing ', 0.00011360188035244962)
quadgram
('tumult ', 0.0018831168831168826)
pentagram
('turn ', 0.10259740259740259)
